In [2]:
import torch
import torch.nn as nn
import numpy as np
import open3d as o3d
from torch.utils.data import DataLoader
from learning3d.models import PPFNet
from learning3d.data_utils import ClassificationData, ModelNet40Data
import random

# Cihaz ayarı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Test veri kümesini yükleme
test_dataset = ClassificationData(data_class=ModelNet40Data(train=False, num_points=4096))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

# Modeli oluştur ve ağırlıkları yükle
class PPFNetClassifier(nn.Module):
    def __init__(self, num_classes=40, embedding_dim=128):
        super(PPFNetClassifier, self).__init__()
        self.encoder = PPFNet(features=['ppf', 'dxyz', 'xyz'], emb_dims=embedding_dim, radius=0.3, num_neighbors=64)
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.4),

            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.4),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3),

            nn.Linear(512, num_classes)
        )
        
    def forward(self, points, normals):
        features = self.encoder(points, normals)
        global_embedding = torch.max(features, dim=1)[0]
        return self.fc(global_embedding)

model = PPFNetClassifier(num_classes=40, embedding_dim=128).to(device)
model.encoder.load_state_dict(torch.load("ppfnet_encoder_epoch25.pth", map_location=device))
model.fc.load_state_dict(torch.load("ppfnet_classifier_epoch25.pth", map_location=device))
model.eval()

# Normal hesaplama fonksiyonu
def compute_normals_single(points_single):
    """Open3D kullanarak yüzey normallerini hesaplar."""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_single.astype(np.float64))
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    return np.asarray(pcd.normals)

# Rastgele dönüşüm uygulama fonksiyonları
def random_rotation_single_axis(points, max_angle):
    """Belirli bir eksende rastgele dönüşüm uygular."""
    axis = random.choice([0, 1, 2])  # Rastgele bir eksen seç
    angle = random.uniform(-max_angle, max_angle)  # Rastgele dönüş açısı belirle
    
    rotation_matrix = torch.eye(3)  # 3x3 birim matris
    c, s = torch.cos(torch.tensor(angle)), torch.sin(torch.tensor(angle))
    
    if axis == 0:  # X ekseni dönüşü
        rotation_matrix[1, 1] = c
        rotation_matrix[1, 2] = -s
        rotation_matrix[2, 1] = s
        rotation_matrix[2, 2] = c
    elif axis == 1:  # Y ekseni dönüşü
        rotation_matrix[0, 0] = c
        rotation_matrix[0, 2] = s
        rotation_matrix[2, 0] = -s
        rotation_matrix[2, 2] = c
    else:  # Z ekseni dönüşü
        rotation_matrix[0, 0] = c
        rotation_matrix[0, 1] = -s
        rotation_matrix[1, 0] = s
        rotation_matrix[1, 1] = c

    return torch.matmul(points, rotation_matrix.to(points.device))  # Dönüşümü uygula


def random_rotation(points, max_angle):
    """Rastgele dönüşüm uygular."""
    angles = torch.empty(3).uniform_(-max_angle, max_angle)  # Her eksen için rastgele açı seç
    Rx = torch.tensor([
        [1, 0, 0],
        [0, torch.cos(angles[0]), -torch.sin(angles[0])],
        [0, torch.sin(angles[0]), torch.cos(angles[0])]
    ])
    Ry = torch.tensor([
        [torch.cos(angles[1]), 0, torch.sin(angles[1])],
        [0, 1, 0],
        [-torch.sin(angles[1]), 0, torch.cos(angles[1])]
    ])
    Rz = torch.tensor([
        [torch.cos(angles[2]), -torch.sin(angles[2]), 0],
        [torch.sin(angles[2]), torch.cos(angles[2]), 0],
        [0, 0, 1]
    ])
    rotation_matrix = torch.matmul(Rz, torch.matmul(Ry, Rx))  # Döndürme matrisini oluştur
    return torch.matmul(points, rotation_matrix.to(points.device))  # Dönüşümü uygula


def evaluate_with_transformation(rotation_type, max_angle):
    correct, total = 0, 0
    with torch.no_grad():
        for data in test_loader:
            points, labels = data
            points = points.to(device, dtype=torch.float32)
            labels = labels.squeeze().to(device)
        
            if rotation_type == "single_small":
                points = random_rotation_single_axis(points, max_angle)  # Tek eksende ufak dönüş
            elif rotation_type == "all_small":
                points = random_rotation(points, max_angle)  # Tüm eksenlerde ufak dönüş
            elif rotation_type == "single_large":
                points = random_rotation_single_axis(points, max_angle)  # Tek eksende büyük dönüş
            elif rotation_type == "all_large":
                points = random_rotation(points, max_angle)  # Tüm eksenlerde büyük dönüş
            
            normals = torch.tensor(np.stack([compute_normals_single(p.cpu().numpy()) for p in points.cpu()], axis=0), dtype=torch.float32).to(device)
            
            outputs = model(points, normals)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    accuracy = 100 * correct / total
    print(f"Test Accuracy ({rotation_type}): {accuracy:.2f}%")

# Test senaryoları
evaluate_with_transformation("single_small", max_angle=torch.pi/18)  # ~10 derece tek eksen
evaluate_with_transformation("all_small", max_angle=torch.pi/18)  # ~10 derece tüm eksenler
evaluate_with_transformation("single_large", max_angle=torch.pi/3)  # ~60 derece tek eksen
evaluate_with_transformation("all_large", max_angle=torch.pi/3)  # ~60 derece tüm eksenler

Using device: cuda
Test Accuracy (single_small): 88.90%
Test Accuracy (all_small): 87.36%
Test Accuracy (single_large): 56.56%
Test Accuracy (all_large): 21.11%
